In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import numpy as np
import re
import os
import requests

In [ ]:
def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

filename = 'WSCC-09_RX'
url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_RX/WSCC-09_RX'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')
download_grid_data(filename+'_DI.xml', url+'_DI.xml')

## Run

In [ ]:
path = 'logs/DP_WSCC9bus_SGReducedOrderVBR/'
logName = 'DP_WSCC9bus_SGReducedOrderVBR'
logFilename = path + logName

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

DP_WSCC9bus_SGReducedOrderVBR ${TOP}/WSCC-09_RX_*.xml -U SparseLU --timestep=10e-6 --duration=0.1

In [ ]:
%%capture
ts_coupled_sparse_rv = rt.read_timeseries_dpsim(logFilename+'_RightVector.csv')
ts_coupled_sparse_lv = rt.read_timeseries_dpsim(logFilename+'_LeftVector.csv')

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

DP_WSCC9bus_SGReducedOrderVBR ${TOP}/WSCC-09_RX_*.xml -U KLU --timestep=10e-6 --duration=0.1

In [ ]:
%%capture
ts_coupled_klu_rv = rt.read_timeseries_dpsim(logFilename+'_RightVector.csv')
ts_coupled_klu_lv = rt.read_timeseries_dpsim(logFilename+'_LeftVector.csv')

## Validate Results

In [ ]:
tolerance = 1e-2

for entry in ts_coupled_sparse_rv:
    sparse_values = ts_coupled_sparse_rv[entry].values
    klu_values = ts_coupled_klu_rv[entry].values
    # 2-norm of errors
    error = np.linalg.norm(sparse_values-klu_values, ord=2)
    assert(error < tolerance)

for entry in ts_coupled_sparse_lv:
    sparse_values = ts_coupled_sparse_lv[entry].values
    klu_values = ts_coupled_klu_lv[entry].values
    # 2-norm of errors
    error = np.linalg.norm(sparse_values-klu_values, ord=2)
    assert(error < tolerance)